In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

/home/ec2-user/anaconda3/envs/new_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Train, Validation, Test Dataset
train_dataset = pd.read_csv('saveDir/LOS_WEEKS_adm_train.csv')
val_dataset = pd.read_csv('saveDir/LOS_WEEKS_adm_val.csv')
test_dataset = pd.read_csv('saveDir/LOS_WEEKS_adm_test.csv')

In [3]:
from transformers import AutoModelForSequenceClassification, AutoConfig

# Specify the dropout rate in the configuration
config = AutoConfig.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1', 
                                    num_labels=4, 
                                    hidden_dropout_prob=0.2, 
                                    attention_probs_dropout_prob=0.2)

# Load the pre-trained model with the specified configuration
core_model = AutoModelForSequenceClassification.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1', config=config)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bvanaken/CORe-clinical-outcome-biobert-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', 
#                                     num_labels=4, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# clinical_model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', config=config)

In [5]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('dmis-lab/biobert-base-cased-v1.2', 
#                                     num_labels=4, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# base_model = AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.2', config=config)

In [6]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT', 
#                                     num_labels=4, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# discharge_model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT', config=config)

In [7]:
from transformers import AutoTokenizer

# Choose a tokenizer
tokenizer = AutoTokenizer.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1')

In [8]:
from transformers import BertTokenizer, BertModel
import torch

def split_inputs(input_ids, attention_mask, tokenizer):
    # Get the special token ids
    cls_token_id = tokenizer.cls_token_id
    sep_token_id = tokenizer.sep_token_id

    # Split the tensors
    input_ids1 = input_ids[:, :510]
    input_ids2 = input_ids[:, 510:1020]

    attention_mask1 = attention_mask[:, :510]
    attention_mask2 = attention_mask[:, 510:1020]

    # Add [CLS] and [SEP] tokens
    input_ids1 = torch.cat([torch.full_like(input_ids1[:, :1], cls_token_id), input_ids1, torch.full_like(input_ids1[:, :1], sep_token_id)], dim=1)
    input_ids2 = torch.cat([torch.full_like(input_ids2[:, :1], cls_token_id), input_ids2, torch.full_like(input_ids2[:, :1], sep_token_id)], dim=1)

    attention_mask1 = torch.cat([torch.full_like(attention_mask1[:, :1], 1), attention_mask1, torch.full_like(attention_mask1[:, :1], 1)], dim=1)
    attention_mask2 = torch.cat([torch.full_like(attention_mask2[:, :1], 1), attention_mask2, torch.full_like(attention_mask2[:, :1], 1)], dim=1)

    return (input_ids1, attention_mask1), (input_ids2, attention_mask2)

In [9]:
from torch.utils.data import DataLoader
from torch import nn

class EnsembleModel(nn.Module):
    def __init__(self, model1):
        super(EnsembleModel, self).__init__()
        self.model1 = model1

    def forward(self, input_ids, attention_mask):
        # Assuming your batch size (18) tensors are input_ids and attention_mask
        (input_ids1, attention_mask1), (input_ids2, attention_mask2) = split_inputs(input_ids, attention_mask, tokenizer)

        outputs1 = self.model1(input_ids1, attention_mask=attention_mask1)[0]
        outputs2 = self.model1(input_ids2, attention_mask=attention_mask2)[0]
        
        avg_output = (outputs1 + outputs2) / 2.00
        return avg_output

In [10]:
# Apply the tokenizer to the training, validation, and test datasets
train_encodings = tokenizer(train_dataset['text'].tolist(), truncation=True, padding=True, max_length = 1020)
val_encodings = tokenizer(val_dataset['text'].tolist(), truncation=True, padding=True,  max_length = 1020)
test_encodings = tokenizer(test_dataset['text'].tolist(), truncation=True, padding=True , max_length = 1020)

In [11]:
# Create a Dataset for PyTorch
class LosDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
train_dataset = LosDataset(train_encodings, train_dataset['los_label'].tolist())
val_dataset = LosDataset(val_encodings, val_dataset['los_label'].tolist())
test_dataset = LosDataset(test_encodings, test_dataset['los_label'].tolist())

In [13]:
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from tqdm import tqdm
from torch import nn
import numpy as np

# Create the ensemble model
ensemble_model = EnsembleModel(core_model)

In [14]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'CORE_baseline'
core_models = [f for f in files if f.startswith('CORE_chunk')]

if core_models:
    print("Found models starting with 'CORE_chunk':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[0]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'CORE_chunk'.")

No models found starting with 'CORE_chunk'.


In [15]:
# ensemble_model

In [16]:
# Push the model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)

In [17]:
train_loader = DataLoader(train_dataset, batch_size=9, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=9, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=9, shuffle=False)

In [18]:
epochs = 200
best_roc_auc = 0.0
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 3
gradient_accumulation_steps = 10

# Set the optimizer
optimizer = AdamW(ensemble_model.parameters(), lr=1e-5, weight_decay=0.01)

# Set the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=50, 
    num_training_steps=len(train_loader) * epochs // gradient_accumulation_steps
)


/home/ec2-user/anaconda3/envs/new_env/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
# device = "cpu"

In [20]:
from torch.nn import functional as F
# Training
for epoch in range(epochs):
    ensemble_model.train()
    train_loss = 0
    for step, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            optimizer.step()
            scheduler.step()

    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
            

    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class, average='weighted')
    precision = precision_score(val_labels, val_preds_class, average='weighted')
    f1 = f1_score(val_labels, val_preds_class, average='weighted')
    micro_f1 = f1_score(val_labels, val_preds_class, average='micro')
    macro_roc_auc = roc_auc_score(val_labels, val_preds, multi_class='ovo', average='macro')

    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')
        
        
    # Implement early stopping
    if epoch > 0 and macro_roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = macro_roc_auc
        early_stopping_count = 0
        torch.save(ensemble_model.state_dict(), f"CORE_chunk_epoch_{epoch}roc_{best_roc_auc}.pth")

100%|█████████████████████████████████████████| 488/488 [04:13<00:00,  1.92it/s]
/home/ec2-user/anaconda3/envs/new_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1/200, Training Loss: 1.3202797396406554, Validation Loss: 1.2764267369371947
Accuracy: 0.3844226827601913, Recall: 0.3844226827601913, Precision: 0.3205198689148802, F1: 0.32180794885618486, Micro F1: 0.3844226827601913, Macro Roc Auc: 0.6192931790018936


100%|█████████████████████████████████████████| 488/488 [04:40<00:00,  1.74it/s]


Epoch: 2/200, Training Loss: 1.2638431598152222, Validation Loss: 1.2423362025960547
Accuracy: 0.3907993623320428, Recall: 0.3907993623320428, Precision: 0.39714591435398944, F1: 0.3475521915602991, Micro F1: 0.3907993623320428, Macro Roc Auc: 0.6575574673385279


100%|█████████████████████████████████████████| 488/488 [04:17<00:00,  1.89it/s]


...

Epoch: 9/200, Training Loss: 1.0660182790303505, Validation Loss: 1.2173509387696375
Accuracy: 0.43179230243680256, Recall: 0.43179230243680256, Precision: 0.42788308628885, F1: 0.42628415567727934, Micro F1: 0.43179230243680256, Macro Roc Auc: 0.7086186621666707
EarlyStopping counter: 1 out of 3


100%|█████████████████████████████████████████| 488/488 [04:34<00:00,  1.78it/s]


Epoch: 10/200, Training Loss: 1.0433854711757402, Validation Loss: 1.2646698060094332
Accuracy: 0.4215440674106126, Recall: 0.4215440674106126, Precision: 0.4200047139497395, F1: 0.4140533040017525, Micro F1: 0.4215440674106126, Macro Roc Auc: 0.7057936108556873
EarlyStopping counter: 2 out of 3


100%|█████████████████████████████████████████| 488/488 [04:39<00:00,  1.75it/s]

Epoch: 11/200, Training Loss: 1.0112699334419044, Validation Loss: 1.2730882198351328
Accuracy: 0.42837622409473924, Recall: 0.42837622409473924, Precision: 0.4288054707737269, F1: 0.4232874936349611, Micro F1: 0.42837622409473924, Macro Roc Auc: 0.7044803901863924
EarlyStopping counter: 3 out of 3
Early stopping


In [24]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'CORE_baseline'
core_models = [f for f in files if f.startswith('CORE_chunk')]

if core_models:
    print("Found models starting with 'CORE_chunk':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[0]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'CORE_chunk'.")

Found models starting with 'CORE_chunk':
CORE_chunk_epoch_7roc_0.7099943517340308.pth
Loaded Model


In [25]:
# Put the model in evaluation mode
ensemble_model.eval()

# Initialize lists to store predictions and true labels
test_preds = []
test_labels = []

# Iterate over test data
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())



100%|█████████████████████████████████████████| 978/978 [08:42<00:00,  1.87it/s]


In [26]:
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

# Calculate metrics
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class, average='weighted')
precision = precision_score(test_labels, test_preds_class, average='weighted')
f1 = f1_score(test_labels, test_preds_class, average='weighted')
micro_f1 = f1_score(test_labels, test_preds_class, average='micro')
macro_roc_auc = roc_auc_score(test_labels, test_preds, multi_class='ovo', average='macro')

print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')

Accuracy: 0.44810730930999204, Recall: 0.44810730930999204, Precision: 0.44439150557468715, F1: 0.43654376862633537, Micro F1: 0.44810730930999204, Macro Roc Auc: 0.7252680623582094
